In [9]:
#!pip install memory_profiler
import time
from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from memory_profiler import profile
from scipy.interpolate import interp1d

source_directory = 'points'
output_csv_directory = 'output_csv'
output_jpg_directory = 'output_jpg'

In [10]:
def parse_xlsx(source_directory, output_csv_directory, output_jpg_directory):
    files = [f for f in listdir(source_directory) if isfile(join(source_directory, f))]

    for curr in files:
        filepath = source_directory + '/' + curr
        df = pd.read_excel(filepath)
        material = df.columns[0]
        df = df.drop(index=0)
        arr = df.to_numpy(dtype='float64')
        temps, index, counts = np.unique(df.values.T[0], return_index=True, return_inverse=False, return_counts=True)
        for i in range(len(temps)):
            start = index[i]
            end = index[i] + counts[i]
            strain = arr.T[1][start:end]
            stress = arr.T[2][start:end]

            # Create a cubic spline interpolation function
            f = interp1d(strain, stress, kind='cubic')

            # Create a vector of x values to interpolate over
            x_interp = np.linspace(strain[0], strain[-1], 1000)

            # Use the spline interpolation function to interpolate the y values
            y_interp = f(x_interp)

            # Write x and y values to a CSV file
            csv_file = f'{output_csv_directory}/{material}_{temps[i]}°C.csv'
            with open(csv_file, 'w') as file:
                file.write('Strain,Stress\n')
                for j in range(len(x_interp)):
                    file.write(f'{x_interp[j]},{y_interp[j]}\n')

            # Plot the interpolated curve and original data points
            plt.plot(x_interp, y_interp)

            # Add axis labels, title, and legend
            plt.xlabel('Strain [%]')
            plt.ylabel('Stress [MPa]')

            # plt.title('Stress-Strain Curve for ~MATERIAL NAME Temperature~')
            title = f'Stress-Strain Curve for {material}/{temps[i]}°C'
            plt.title(title)

            image_path = f'{output_jpg_directory}/{material}_{temps[i]}°C.jpg'
            plt.savefig(image_path)

            plt.clf()

In [11]:
start = time.time_ns()

parse_xlsx(source_directory, output_csv_directory, output_jpg_directory)

end = time.time_ns()

<Figure size 432x288 with 0 Axes>

In [12]:
print("Runtime for 2 materials (5 outputs in total): ")
print(float(end - start) / (10 ** 9))

Runtime for 2 materials (5 outputs in total): 
0.2769713


In [13]:
!python -m memory_profiler parse_xlsx.py

Filename: parse_xlsx.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    15    105.9 MiB    105.9 MiB           1   @profile
    16                                         def parse_xlsx(source_directory, output_csv_directory, output_jpg_directory):
    17    105.9 MiB      0.0 MiB           5       files = [f for f in listdir(source_directory) if isfile(join(source_directory, f))]
    18                                         
    19    124.8 MiB      0.0 MiB           3       for curr in files:
    20    123.3 MiB      0.0 MiB           2           filepath = source_directory + '/' + curr
    21    123.5 MiB     11.6 MiB           2           df = pd.read_excel(filepath)
    22    123.5 MiB      0.0 MiB           2           material = df.columns[0]
    23    123.5 MiB      0.1 MiB           2           df = df.drop(index=0)
    24    123.5 MiB      0.0 MiB           2           arr = df.to_numpy(dtype='float64')
    25    123.5 MiB      0.0 MiB           2        

from this output, we can determine that our program consumes 120 MiB during runtime